In [1]:
import helper
import matplotlib.pyplot as plt
from keras.applications import *
from keras.callbacks import EarlyStopping
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
#设置各种参数
train_path = ['./data/train2/cat', './data/train2/dog']
test_path ='./data/test1/test1'
img_size =(299,299)
layer_num = 630
model_image ='./models/model_image_fine_tuning_inceptionresnetv2_0402_nu.png'
model_weights_file = './models/weights_fine_tuning_inceptionresnetv2_with_outliers_0402_nu.h5'
template_csv_path = './predicts/sample_submission.csv'
target_csv_path = './predicts/pred_fine_tuning_inceptionresnetv2_with_outliers_0402_nu.csv'
MODEL = inception_resnet_v2.InceptionResNetV2
preprocess_func = inception_resnet_v2.preprocess_input

In [4]:
#获取训练集数据
X_train, y_train, image_files= helper.get_train_input_from_folder_with_subclasses(train_path, img_size, lambda_func=preprocess_func)
print("finished")

(24959, 299, 299, 3)
(24959, 1)
finished
finished


In [5]:
#构造模型，锁定base_model所有层
model = helper.get_fine_tuning_first_model(MODEL)

#可视化模型
helper.visualize_model(model, model_image)
print("finished")

start
(?, ?, ?, 3)
(?, ?, ?, 1536)
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.517034 to fit

finished


In [5]:
print("start")
#第一次训练新添加层权重
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 19967 samples, validate on 4992 samples
Epoch 1/8
19967/19967 [==============================] - 133s 7ms/step - loss: 0.2058 - acc: 0.9359 - val_loss: 0.0791 - val_acc: 0.9822
Epoch 2/8
19967/19967 [==============================] - 123s 6ms/step - loss: 0.0866 - acc: 0.9741 - val_loss: 0.0568 - val_acc: 0.9872
Epoch 3/8
19967/19967 [==============================] - 124s 6ms/step - loss: 0.0740 - acc: 0.9759 - val_loss: 0.0884 - val_acc: 0.9730
Epoch 4/8
19967/19967 [==============================] - 124s 6ms/step - loss: 0.0660 - acc: 0.9778 - val_loss: 0.0403 - val_acc: 0.9898
Epoch 5/8
19967/19967 [==============================] - 124s 6ms/step - loss: 0.0668 - acc: 0.9771 - val_loss: 0.0670 - val_acc: 0.9804
Epoch 6/8
19967/19967 [==============================] - 124s 6ms/step - loss: 0.0602 - acc: 0.9799 - val_loss: 0.0495 - val_acc: 0.9862
Epoch 7/8
19967/19967 [==============================] - 124s 6ms/step - loss: 0.0572 - acc: 0.9791 - val_loss: 0.0795 - va

In [6]:
print("start")
#放开若干层权重，再次训练
model = helper.get_fine_tuning_second_model(model, layer_num)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks=[early_stopping])
print("finished")

start
Train on 19967 samples, validate on 4992 samples
Epoch 1/50
19967/19967 [==============================] - 146s 7ms/step - loss: 0.0573 - acc: 0.9805 - val_loss: 0.0237 - val_acc: 0.9960
Epoch 2/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0563 - acc: 0.9800 - val_loss: 0.0215 - val_acc: 0.9960
Epoch 3/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0504 - acc: 0.9834 - val_loss: 0.0200 - val_acc: 0.9962
Epoch 4/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0494 - acc: 0.9824 - val_loss: 0.0190 - val_acc: 0.9960
Epoch 5/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0477 - acc: 0.9836 - val_loss: 0.0181 - val_acc: 0.9964
Epoch 6/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0462 - acc: 0.9839 - val_loss: 0.0175 - val_acc: 0.9964
Epoch 7/50
19967/19967 [==============================] - 135s 7ms/step - loss: 0.0453 - acc: 0.9836 - val_loss: 0.01

In [7]:
#保存模型参数
model.save_weights(model_weights_file)
del X_train
del y_train
print("finished")

finished


In [8]:
print("start")
#获取测试数据和对应文件
X_test, test_files = helper.get_input_from_folder_with_image_files(test_path, img_size, lambda_func=preprocess_func)

#获取文件basename
image_file_names = [os.path.basename(path) for path in test_files]

#预测并保存预测结果到csv
helper.predict_and_update_to_csv(model, X_test, image_file_names, template_csv_path, target_csv_path)

print("finished")

start
finished
12500/12500 [==============================] - 71s 6ms/step
finished
finished
